In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install pmdarima

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Data Visualization 

import seaborn as sns
import plotly.express as px
%matplotlib inline 

# model building packages 

from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace import sarimax
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.metrics import mean_absolute_error

In [ ]:
df_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
category = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')

In [ ]:
category.head()

In [ ]:
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')

In [ ]:
items.head()

In [ ]:
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
shops.head()

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
test.head()

In [ ]:
df_train.info()

In [ ]:
df_train.columns

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.describe().round(2)

In [ ]:
print('Date Range in Train Dataset -', df_train['date'].min(), '-' ,df_train['date'].max())

In [ ]:
corr = df_train.corr()

In [ ]:
corr

In [ ]:
fig = px.imshow(corr, zmin=-1, zmax=1)
fig.show()

In [ ]:
Q1 = df_train.quantile(0.25)
Q3 = df_train.quantile(0.75)
IQR = Q3-Q1

((df_train<(Q1-1.5*IQR)) | (df_train>(Q3+1.5*IQR))).sum()

In [ ]:
from pylab import rcParams
rcParams['figure.figsize']=20,8

sns.boxplot(df_train['item_price'], orient='h')

In [ ]:
sns.boxplot(df_train['item_cnt_day'], orient='h');

In [ ]:
df_train = df_train.drop(df_train[df_train['item_price']>=45000].index)
df_train = df_train.drop(df_train[df_train['item_price']<0].index)

In [ ]:
df_train = df_train.drop(df_train[df_train['item_cnt_day']<0].index)
df_train=df_train.drop(df_train[df_train['item_cnt_day']>=600].index)

In [ ]:
df_train.shape

# Map Item Category ID based on Item_ID

In [ ]:
def match_key(key):
    return item_id[key]

In [ ]:
item_id = {key:value for key, value in zip(items['item_id'], items['item_category_id'])}
print("item_id size : {}".format(len(item_id)))

In [ ]:
df_train['item_category_id'] = df_train.apply(lambda x : match_key(x['item_id']), axis=1)
df_train.head()

In [ ]:
df_train['sales']= df_train['item_price']*df_train['item_cnt_day']
df_train.head()

In [ ]:
df_train.groupby(['date_block_num'])['sales'].sum()

# Pandas Profiling

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
ProfileReport(df_train)

In [ ]:
df_train = df_train.drop_duplicates(['date_block_num','shop_id','item_id'])
print("train data Shape : {}".format(df_train.shape))
df_train.head()

In [ ]:
sns.pairplot(df_train)

# Model Building

In [ ]:
SD = seasonal_decompose(df_train['sales'].values, period=12)

SD.plot();

In [ ]:
SD1 = seasonal_decompose(df_train['item_cnt_day'].values, period=12)

SD1.plot();

In [ ]:
def ADF_test(series):
    print('ADFuller Test Results')
    res = adfuller(series, autolag='AIC')
    
    labels = ['ADF Test Statistic', 'P-Value', '# Lags Used', '# Observations']
    
    out = pd.Series(res[0:4], index= labels)
    
    for key, val in res[4].items():
        out[f'Critical value ({key})']= val
        
    print(out)
    
    if res[1]<0.05:
        print('Fail to Reject Null - Data is Stationary')
        
    else:
        print('Reject Null - Data is non-stationary')